# Домашнее задание № 7

### Задание 1 Реализовать алгоритм Леска и проверить его на реальном датасете (8 баллов)

Ворднет можно использовать для дизамбигуации. Самый простой алгоритм дизамбигуации - алгоритм Леска. В нём нужное значение слова находится через пересечение слов контекста, в котором употреблено это слово, с определениями значений слова из ворднета. Значение с максимальным пересечением - нужное.

In [ ]:
import numpy as np

import nltk
nltk.download('punkt')
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize  
from string import punctuation

from nltk.corpus import wordnet as wn

### Реализация алгоритма Леска 

In [75]:
wordnet_lemmatizer = WordNetLemmatizer()

def normalize(text):
  tokens = word_tokenize(text)
  tokens = [wordnet_lemmatizer.lemmatize(t) for t in tokens if t not in punctuation]   

  return tokens

In [73]:
# в качестве контекста будем рассматривать не более 10-ти ближайших слов 
def get_words_in_context(word, sentence, window=5):
  padding = ['PAD'] *  window
  if type(sentence) == type(''):
    sentence = normalize(sentence)
  # добавляем 'PAD' в начало предложения, чтобы избежать IndexError-а
  sentence = padding + sentence 
  sentence = sentence + padding # и в конец предложения тоже

  word_index = sentence.index(word) # получаем индекс слова в предложении
  context = []
  # задаем контекст
  context.extend(sentence[word_index-window:word_index])
  context.extend(sentence[word_index+1:word_index+1+window])
  
  return list(filter(('PAD').__ne__, context)) # удаляем 'PAD'-ы, если есть

In [53]:
get_words_in_context('club', 'welcome to the club')

['welcome', 'to', 'the']

In [54]:
get_words_in_context('shop', 'i went to the shop yesterday and bought some food')

['i', 'went', 'to', 'the', 'yesterday', 'and', 'bought', 'some', 'food']

In [55]:
get_words_in_context('dog', 'your dog is sleeping')

['your', 'is', 'sleeping']

In [76]:
def lesk(word, sentence):
    bestsense = 0
    maxoverlap = 0
    synsets = wn.synsets(word)
    context = set(get_words_in_context(word, sentence))
    
    for i,syn in enumerate(synsets):
      definition = set(normalize(syn.definition()))
      # находим пересечение контекста и определения
      intersection = definition.intersection(context)
      if len(intersection) > maxoverlap:
        maxoverlap = len(intersection)
        bestsense = i 
    return bestsense

**Note:** так как контекст преобразуется в set, повторы слов не будут учитываться. То есть, если контект будет "home sweet home", опрелению со словом 'home' прибавится только 1 балл. 

In [45]:
lesk('day', 'a day is some point or period in time')

1

In [26]:
wn.synsets('day')[1].definition()

'some point or period in time'

In [46]:
lesk('fall', 'my favourite season is fall')

0

In [28]:
wn.synsets('fall')[0].definition()

'the season when the leaves fall from the trees'

In [47]:
lesk('falling', 'Huge drops of rain were falling from the sky.')

0

In [30]:
wn.synsets('falling')[0].definition()

'descend in free fall under the influence of gravity'

### Оценка качества алгоритма на датасете для задач WSD

Загружаем корпус:

In [112]:
corpus_wsd = []
corpus = open('corpus_wsd_50k.txt').read().split('\n\n')
for sent in corpus:
    corpus_wsd.append([s.split('\t') for s in sent.split('\n')])

Возьмем первые 10 000 предложений

In [114]:
corpus_wsd = corpus_wsd[0:10000]

Корпус состоит из предложений, где у каждого слова три поля - значение, лемма и само слово. Значение пустое, когда слово однозначное, а у многозначных слов стоит тэг вида **'long%3:00:02::'** Это тэг wordnet'ного формата

In [120]:
corpus_wsd[1]

[['', 'have', 'Have'],
 ['', 'you', 'you'],
 ['permit%2:41:00::', 'permit', 'permitted'],
 ['', 'it', 'it'],
 ['', 'to', 'to'],
 ['become%2:42:01::', 'become', 'become'],
 ['', 'a', 'a'],
 ['giveaway%1:21:00::', 'giveaway', 'giveaway'],
 ['program%1:09:01::', 'program', 'program'],
 ['rather%4:02:02::', 'rather', 'rather'],
 ['', 'than', 'than'],
 ['', 'one', 'one'],
 ['', 'that', 'that'],
 ['have%2:42:00::', 'have', 'has'],
 ['', 'the', 'the'],
 ['goal%1:09:00::', 'goal', 'goal'],
 ['', 'of', 'of'],
 ['improved%3:00:00::', 'improved', 'improved'],
 ['employee%1:18:00::', 'employee', 'employee'],
 ['morale%1:26:00::', 'morale', 'morale'],
 ['', 'and', 'and'],
 ['', ',', ','],
 ['consequently%4:02:00::', 'consequently', 'consequently'],
 ['', ',', ','],
 ['increased%3:00:00::', 'increased', 'increased'],
 ['productivity%1:07:00::', 'productivity', 'productivity'],
 ['', '?', '?']]

In [115]:
comparison_list_bool = []

for sent in corpus_wsd:
  sent_joined = [word[1] for word in sent]
  for word in sent:
    # рассматриваем только многозначные слова
    if word[0] != '':
      lesk_index = lesk(word[1], sent_joined)
      is_equal = wn.synsets(word[1])[lesk_index] == wn.lemma_from_key(word[0]).synset()
      comparison_list_bool.append(int(is_equal))

N = len(comparison_list_bool)
print(f'Число проверенных слов: {N}')
print(f'Accuracy: {round(np.array(comparison_list_bool).sum() / N,3)}')

Число проверенных слов: 93950
Accuracy: 0.407


**Вывод:** алгоритм Леска показал достаточно низкую точность на предложенном корпусе: **0.407**. Возможно, определенные модификации позволят улучшить качество данного метода. 